In [2]:
import re
import httpx
import os
import openai
import gsearch
import slcindex
from datetime import datetime
import callgpt
import ReAct_chain
import langchain
import chromadb

def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()


os.environ["OPENAI_API_KEY"] = open_file('openai_api_key.txt')
openai.api_key = open_file('openai_api_key.txt')
openai_api_key = openai.api_key

In [2]:
SYSTEM_INSTRUCTION = """"You are slcbot, Rohit Krishnan's personal representative and a world-class expert at everything. You use both the context and your world knowledge to answer questions. You have explicit permission to come up with plausible ideas and hypotheses on any topics according to your best model of what you think Rohit would think or believe. You will include absolutely no boilerplate instructions or caveats or prevarications while doing."""

CONTEXT_UNIVERSAL = """Current date/time: {datetime}"""

BASIC_INFO = ""#"""CONTEXT. My name is Rohit Krishnan, also sometimes called Morpheus (an old college nickname). I have a wife and 2 kids. I live in London, and used to live in Singapore and India before. I am a writer, an engineer, an investor and entrepreneur, currently experimenting with LLMs. I write essays at https://www.strangeloopcanon.com and my twitter username is @krishnanrohit."""

In [3]:
class Thought:
    def __init__(self):
        self.messages = []
        self.messages.append({"role": "system", "content": "SYSTEM_INSTRUCTION"})
        self.messages.append({"role": "system", "content": "BASIC_INFO"})
        self.messages.append(
            {"role": "user", "content": CONTEXT_UNIVERSAL.format(datetime=datetime.now())})
        
    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

ask = callgpt.Ask()
def get_tasks(user_input):
    tasks = ask.route_user_message(user_input)
    return tasks

def get_task_list(task):
        messages = []
        Thought()
        messages.append({"role": "user", "content": "Please create an explicit list of maximum 3 instructions on how you might go about finding the answer to this query using Wikipedia and Google searches"+task})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        model_response = response.choices[0].message.content
        return model_response

def task_prio(tasks):
        messages = []
        Thought()
        messages.append({"role": "user", "content": "Looking at the steps laid out here, what would be your first task? "+tasks})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        model_response = response.choices[0].message.content
        return model_response

In [8]:
question = get_tasks("What's the basic theory of French political economy?")
tasks = get_task_list(question)
print("tasklist is:"+tasks)
t1 = task_prio(tasks)
print("prioritised task is:" +t1)

tasklist is:1. Use Wikipedia to search for "French political economy" and read through the history section to gain an understanding of the different theories and ideas that have influenced it over time.

2. Use Google searches to explore more in-depth information about the different economic theories mentioned in the Wikipedia article, such as "Mercantilism," "Physiocrats," "classical economists," and "Structuralism." Look for credible sources such as academic journals or reputable websites.

3. Use Google Scholar to search for academic articles or books about French political economy or specific economic theories mentioned in the Wikipedia article for a deeper understanding of the subject matter.
prioritised task is:My first task would be to complete task 1, which is to use Wikipedia to gain a general understanding of the different theories and ideas that have influenced French political economy over time. This would provide a foundation for exploring more in-depth information and con

In [2]:
import sys
import os
import openai
import math
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()
os.environ["OPENAI_API_KEY"] = open_file('openai_api_key.txt')
openai.api_key = open_file('openai_api_key.txt')
openai_api_key = openai.api_key
os.environ["GOOGLE_CSE_ID"] = open_file('google_searchengine_id.txt')
os.environ["GOOGLE_API_KEY"] = open_file('google_api_key.txt')
os.environ["SERPER_API_KEY"] = open_file('serpapikey.txt')
os.environ["WOLFRAM_ALPHA_APPID"] = open_file('wolfram_key.txt')
os.environ["ZAPIER_NLA_API_KEY"] = open_file('zapierkey.txt')
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.utilities import GoogleSearchAPIWrapper, PythonREPL, RequestsWrapper
from langchain.utilities.wolfram_alpha import WolframAlphaAPIWrapper
from langchain.agents import load_tools, initialize_agent
from langchain.agents.agent_toolkits import ZapierToolkit
from langchain.utilities.zapier import ZapierNLAWrapper
search = GoogleSearchAPIWrapper()
requests = RequestsWrapper()
wolfram = WolframAlphaAPIWrapper()
zapier = ZapierNLAWrapper()
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
agent = initialize_agent(toolkit.get_tools(), llm, agent="zero-shot-react-description", verbose=True)
python_repl = PythonREPL()
llm = OpenAI(temperature=0)
tools = load_tools(["google-serper", "google-search", "python_repl"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run(t1)

'Assumption: convert 1 oz of gold (chemical element) to US nickels \nAnswer: 35947.80 US nickels'

In [ ]:
answer = ReAct_chain.query(t1)
print(answer)

In [ ]:
# tasklength = t1.split('\n')
# for i in len(tasklength):
#     ReAct_chain.query(tasklength[i])
    